In [ ]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.8 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=68607a1be47bfc8308c292b3b97298ad90cf49a549a78e62c9186e0ee5c58e15
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
import torch
torch.cuda.is_available(), torch.cuda.device_count()

(True, 1)

In [ ]:
!pip install transformers
!pip install datasets
import warnings
warnings.filterwarnings('ignore')

import pickle
import numpy as np
import transformers
from transformers import Trainer
from datasets import load_metric
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 6.8 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 6.6 MB 48.5 MB/s 
     |████████████████████████████████| 596 kB 64.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 7.5 MB/s 
     |████████████████████████████████| 140 kB 76.9 MB/s 
     |████████████████████████████████| 212 kB 66.4 MB/s 
     |████████████████████████████████| 86 kB 8.0 MB/s 
     |████████████████████████████████| 1.1 MB 59.9 MB/s 
     |████████████████████████████████| 127 kB 76.4 MB/s 
     |████████████████████████████████| 271 kB 72.0 MB/s 
     |███████████████

## Всякое


In [ ]:
label_names =  ["O","I-AGE","B-AGE", "B-AWARD",
  "I-AWARD",
  "B-CITY",
  "I-CITY",
  "B-COUNTRY",
  "I-COUNTRY",
  "B-CRIME",
  "I-CRIME",
  "B-DATE",
  "I-DATE",
  "B-DISEASE",
  "I-DISEASE",
  "B-DISTRICT",
  "I-DISTRICT",
  "B-EVENT",
  "I-EVENT",
  "B-FACILITY",
  "I-FACILITY",
  "B-FAMILY",
  "I-FAMILY",
  "B-IDEOLOGY",
  "I-IDEOLOGY",
  "B-LANGUAGE",
  "I-LAW",
  "B-LAW",
  "B-LOCATION",
  "I-LOCATION",
  "B-MONEY",
  "I-MONEY",
  "B-NATIONALITY",
  "I-NATIONALITY",
  "B-NUMBER",
  "I-NUMBER",
  "B-ORDINAL",
  "I-ORDINAL",
  "B-ORGANIZATION",
  "I-ORGANIZATION",
  "B-PENALTY",
  "I-PENALTY",
  "B-PERCENT",
  "I-PERCENT",
  "B-PERSON",
  "I-PERSON",
  "I-PRODUCT",
  "B-PRODUCT",
  "B-PROFESSION",
  "I-PROFESSION",
  "B-RELIGION",
  "I-RELIGION",
  "B-STATE_OR_PROVINCE",
  "I-STATE_OR_PROVINCE",
  "B-TIME",
  "I-TIME",
  "B-WORK_OF_ART",
  "I-WORK_OF_ART"]

def align_labels_with_tokens(labels: list, word_ids: list) -> list:
    return [-100 if i is None else labels[i] for i in word_ids]

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["sequences"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ids"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [[label_names[p] for (p, l) in zip(prediction, label) if l != -100]
                        for prediction, label in zip(predictions, labels)
                        ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

## Загружаем данные


In [ ]:
raw_datasets = load_dataset("surdan/nerel_short")

Downloading:   0%|          | 0.00/924 [00:00<?, ?B/s]

Using custom data configuration surdan--for_test_v3-eb84b899e14cc88e


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/surdan___parquet/surdan--for_test_v3-eb84b899e14cc88e/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sequences', 'ids'],
        num_rows: 2508
    })
    test: Dataset({
        features: ['sequences', 'ids'],
        num_rows: 512
    })
    dev: Dataset({
        features: ['sequences', 'ids'],
        num_rows: 536
    })
})

## Подготовка данных

In [ ]:
model_checkpoint = "cointegrated/rubert-tiny2"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/401 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Parameter 'function'=<function tokenize_and_align_labels at 0x7f1bf036c050> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2508
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 512
    })
    dev: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 536
    })
})

## Подготовка обучения


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
id2label = {
    "0": "O",
    "1": "I-AGE",
    "2": "B-AGE",
    "3": "B-AWARD",
    "4": "I-AWARD",
    "5": "B-CITY",
    "6": "I-CITY",
    "7": "B-COUNTRY",
    "8": "I-COUNTRY",
    "9": "B-CRIME",
    "10": "I-CRIME",
    "11": "B-DATE",
    "12": "I-DATE",
    "13": "B-DISEASE",
    "14": "I-DISEASE",
    "15": "B-DISTRICT",
    "16": "I-DISTRICT",
    "17": "B-EVENT",
    "18": "I-EVENT",
    "19": "B-FACILITY",
    "20": "I-FACILITY",
    "21": "B-FAMILY",
    "22": "I-FAMILY",
    "23": "B-IDEOLOGY",
    "24": "I-IDEOLOGY",
    "25": "B-LANGUAGE",
    "26": "I-LAW",
    "27": "B-LAW",
    "28": "B-LOCATION",
    "29": "I-LOCATION",
    "30": "B-MONEY",
    "31": "I-MONEY",
    "32": "B-NATIONALITY",
    "33": "I-NATIONALITY",
    "34": "B-NUMBER",
    "35": "I-NUMBER",
    "36": "B-ORDINAL",
    "37": "I-ORDINAL",
    "38": "B-ORGANIZATION",
    "39": "I-ORGANIZATION",
    "40": "B-PENALTY",
    "41": "I-PENALTY",
    "42": "B-PERCENT",
    "43": "I-PERCENT",
    "44": "B-PERSON",
    "45": "I-PERSON",
    "46": "I-PRODUCT",
    "47": "B-PRODUCT",
    "48": "B-PROFESSION",
    "49": "I-PROFESSION",
    "50": "B-RELIGION",
    "51": "I-RELIGION",
    "52": "B-STATE_OR_PROVINCE",
    "53": "I-STATE_OR_PROVINCE",
    "54": "B-TIME",
    "55": "I-TIME",
    "56": "B-WORK_OF_ART",
    "57": "I-WORK_OF_ART"
  }
label2id = {
    "B-AGE": "2",
    "B-AWARD": "3",
    "B-CITY": "5",
    "B-COUNTRY": "7",
    "B-CRIME": "9",
    "B-DATE": "11",
    "B-DISEASE": "13",
    "B-DISTRICT": "15",
    "B-EVENT": "17",
    "B-FACILITY": "19",
    "B-FAMILY": "21",
    "B-IDEOLOGY": "23",
    "B-LANGUAGE": "25",
    "B-LAW": "27",
    "B-LOCATION": "28",
    "B-MONEY": "30",
    "B-NATIONALITY": "32",
    "B-NUMBER": "34",
    "B-ORDINAL": "36",
    "B-ORGANIZATION": "38",
    "B-PENALTY": "40",
    "B-PERCENT": "42",
    "B-PERSON": "44",
    "B-PRODUCT": "47",
    "B-PROFESSION": "48",
    "B-RELIGION": "50",
    "B-STATE_OR_PROVINCE": "52",
    "B-TIME": "54",
    "B-WORK_OF_ART": "56",
    "I-AGE": "1",
    "I-AWARD": "4",
    "I-CITY": "6",
    "I-COUNTRY": "8",
    "I-CRIME": "10",
    "I-DATE": "12",
    "I-DISEASE": "14",
    "I-DISTRICT": "16",
    "I-EVENT": "18",
    "I-FACILITY": "20",
    "I-FAMILY": "22",
    "I-IDEOLOGY": "24",
    "I-LAW": "26",
    "I-LOCATION": "29",
    "I-MONEY": "31",
    "I-NATIONALITY": "33",
    "I-NUMBER": "35",
    "I-ORDINAL": "37",
    "I-ORGANIZATION": "39",
    "I-PENALTY": "41",
    "I-PERCENT": "43",
    "I-PERSON": "45",
    "I-PRODUCT": "46",
    "I-PROFESSION": "49",
    "I-RELIGION": "51",
    "I-STATE_OR_PROVINCE": "53",
    "I-TIME": "55",
    "I-WORK_OF_ART": "57",
    "O": "0"
  }

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Downloading:   0%|          | 0.00/715 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized 

In [ ]:
model.config.num_labels

58

In [ ]:
args = TrainingArguments(
    "ruBert_WITH_nerel",
    evaluation_strategy="epoch",
    save_strategy="steps",
    num_train_epochs=15,
    push_to_hub=False,
    per_device_train_batch_size = 32 ## depending on the total volume of memory of your GPU
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


## Обучение

In [ ]:
## for compute_metrics function
metric = load_metric("seqeval")

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

***** Running training *****
  Num examples = 2508
  Num Epochs = 15
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1185


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.011985,0.479684,0.466271,0.472882,0.758552
2,No log,0.943160,0.489708,0.498951,0.494287,0.769971
3,No log,0.901785,0.499330,0.521321,0.510089,0.777694
4,No log,0.873245,0.497077,0.534953,0.515320,0.781673
5,No log,0.849695,0.517029,0.527962,0.522438,0.786127
6,No log,0.836614,0.508483,0.549983,0.528419,0.787691
7,0.692200,0.828617,0.516746,0.552779,0.534155,0.790913
8,0.692200,0.824194,0.517043,0.564663,0.539805,0.791718
9,0.692200,0.820651,0.519075,0.561167,0.539301,0.794182
10,0.692200,0.816646,0.523292,0.563439,0.542624,0.794892


***** Running Evaluation *****
  Num examples = 512
  Batch size = 8
***** Running Evaluation *****
  Num examples = 512
  Batch size = 8
***** Running Evaluation *****
  Num examples = 512
  Batch size = 8
***** Running Evaluation *****
  Num examples = 512
  Batch size = 8
***** Running Evaluation *****
  Num examples = 512
  Batch size = 8
***** Running Evaluation *****
  Num examples = 512
  Batch size = 8
Saving model checkpoint to ruBert_WITH_nerel/checkpoint-500
Configuration saved in ruBert_WITH_nerel/checkpoint-500/config.json
Model weights saved in ruBert_WITH_nerel/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ruBert_WITH_nerel/checkpoint-500/tokenizer_config.json
Special tokens file saved in ruBert_WITH_nerel/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 512
  Batch size = 8
***** Running Evaluation *****
  Num examples = 512
  Batch size = 8
***** Running Evaluation *****
  Num examples = 512
  Batch size = 8
*****

TrainOutput(global_step=1185, training_loss=0.5271965831643921, metrics={'train_runtime': 89.1496, 'train_samples_per_second': 421.987, 'train_steps_per_second': 13.292, 'total_flos': 80415528308880.0, 'train_loss': 0.5271965831643921, 'epoch': 15.0})

In [ ]:
trainer.save_model("ruBert_WITH_nerel")

Saving model checkpoint to ruBert_WITH_nerel
Configuration saved in ruBert_WITH_nerel/config.json
Model weights saved in ruBert_WITH_nerel/pytorch_model.bin
tokenizer config file saved in ruBert_WITH_nerel/tokenizer_config.json
Special tokens file saved in ruBert_WITH_nerel/special_tokens_map.json
